In [1]:
import sys

sys.path.insert(0,'/home/mmo-gog/dpsim/build/')
sys.path.insert(0,'/home/mmo-gog/dpsim/python/src/dpsim')

import iaewreader
import dpsimpy

from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt

In [2]:
path_file = '/home/mmo-gog/dpsim/examples/Notebooks/9BusValidations/Static_Dynamic_Data_9Bus_System.mat'

Power Flow Simulation

In [3]:
sim_name_pf = '9Bus_SYN_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader = iaewreader.Reader(mpc_file_path=path_file, mpc_name='case9_static_only_SYN')
mpc_reader.load_mpc(domain=iaewreader.Domain.PF)
system_pf = mpc_reader.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

dpsim_result_file = 'logs/' + sim_name_pf + '/' + sim_name_pf + '.csv'
ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

dpsim_results = pd.DataFrame(columns=['Bus', 'V_mag(pu)', 'V_angle(deg)', 'P(MW)', 'Q(MVAr)'])
base_power = 1 #mw
for i in range(len(system_pf.nodes)):
    node_name = system_pf.nodes[i].name() #ex. N5
    node_number = node_name.replace('N', '')
    node_baseV = mpc_reader.mpc_bus_data.loc[mpc_reader.mpc_bus_data['bus_i'] == int(node_number), 'baseKV'].iloc[0] * 1e3
    w_mw = 1e-6
    dpsim_results.loc[i] = ([node_name] + [round(np.absolute(ts_dpsim_pf[node_name + '.V'].values[-1]) / node_baseV, 3)]
        + [round(np.angle(ts_dpsim_pf[node_name + '.V'].values[-1])*180/np.pi, 3)] 
        + [round(w_mw * np.real(ts_dpsim_pf[node_name + '.S'].values[-1]) / base_power, 2)] 
        + [round(w_mw * np.imag(ts_dpsim_pf[node_name + '.S'].values[-1]) / base_power, 2)])

dpsim_results

column number: 24
results length: 2
real column names: []
complex column names: ['N1.S', 'N1.V', 'N10.S', 'N10.V', 'N11.S', 'N11.V', 'N12.S', 'N12.V', 'N2.S', 'N2.V', 'N3.S', 'N3.V', 'N4.S', 'N4.V', 'N5.S', 'N5.V', 'N6.S', 'N6.V', 'N7.S', 'N7.V', 'N8.S', 'N8.V', 'N9.S', 'N9.V']


[10:17:47.634629 9Bus_SYN_PF info] Initialize simulation: 9Bus_SYN_PF
[10:17:47.635663 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N4
[10:17:47.635683 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N5
[10:17:47.635696 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N6
[10:17:47.635707 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N7
[10:17:47.635718 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N8
[10:17:47.635727 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N9
[10:17:47.635737 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N10
[10:17:47.635749 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N11
[10:17:47.635760 9Bus_SYN_PF_PF_Solver warning] Unable to get base voltage at N12
[10:17:47.636389 9Bus_SYN_PF info] Scheduling tasks.
[10:17:47.637155 9Bus_SYN_PF info] Scheduling done.
[10:17:47.637161 9Bus_SYN_PF info] Opening interfaces.
[10:17:47.637164 9Bus_SYN_PF info] Sta

,Bus,V_mag(pu),V_angle(deg),P(MW),Q(MVAr)
0,N1,1.000,0.000,474.43,86.89
1,N2,1.100,-8.001,120.00,189.32
2,N3,1.000,-7.171,120.00,10.74
3,N4,1.044,-3.103,-0.00,0.00
4,N5,0.998,-8.729,-0.00,0.00
5,N6,0.997,-9.193,0.00,0.00
6,N7,0.979,-14.072,-0.00,0.00
7,N8,1.050,-9.747,0.00,0.00
8,N9,1.031,-9.042,-0.00,0.00
9,N10,0.981,-12.166,-200.00,-50.00


SP dynamic simulation

In [4]:
sim_name_dyn_sp = 'SP_only_SYN_case9_dyn'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_sp)

mpc_reader = iaewreader.Reader(mpc_file_path=path_file, mpc_name='case9_static_only_SYN', 
                             mpc_dyn_file_path=path_file, mpc_dyn_name='case9_dyn_only_SYN')
mpc_reader.create_dpsim_objects(domain=iaewreader.Domain.SP, frequency=50, with_avr=False, with_tg=True, with_pss=True)
#create a switch
sw = dpsimpy.sp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
switch_closed = 0.2
sw.set_parameters(1e18, switch_closed)
sw.open()
mpc_reader.dpsimpy_comp_dict['Fault'] = [sw]
#Node 11 is the load at 100kV, node 7 ist a node of 400kV grid, between Node 7 and 11 there is a transformer
mpc_reader.dpsimpy_comp_dict['Fault'].append([dpsimpy.sp.SimNode.gnd, mpc_reader.dpsimpy_busses_dict["N11"]])

# create dpsim topology
system_dyn = mpc_reader.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader.system
system_dyn.init_with_powerflow(system_pf)

logger = dpsimpy.Logger(sim_name_dyn_sp)
for node in system_dyn.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)

# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_dyn_sp, dpsimpy.LogLevel.info)
sim.set_system(system_dyn)
sim.set_time_step(1e-3)
sim.set_final_time(10)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.SparseLU)
sim.do_init_from_nodes_and_terminals(True)
sim.do_system_matrix_recomputation(True)
sim.add_logger(logger)

# add event
sw_event_1 = dpsimpy.event.SwitchEvent(0.2, sw, True)
sw_event_2 = dpsimpy.event.SwitchEvent(0.3, sw, False)
sim.add_event(sw_event_1)
sim.add_event(sw_event_2)

sim.run()

   bus  model       BaseS    H    D        Xd        Xq  Xd_t  Xq_t  Xd_s  \
0  1.0    2.0  670.820393  2.0  0.1  1.811215  1.646559   0.3   0.4   0.2   
1  2.0    2.0  500.000000  2.0  0.1  1.811215  1.646559   0.3   0.4   0.2   
2  3.0    2.0  500.000000  2.0  0.1  1.811215  1.646559   0.3   0.4   0.2   

   Xq_s  Td0_t  Tq0_t  Td0_s  Tq0_s   Ra  
0   0.2    7.0    1.5   0.05   0.05  0.0  
1   0.2    7.0    1.5   0.05   0.05  0.0  
2   0.2    7.0    1.5   0.05   0.05  0.0  
   bus  Type     Ka    Te   Ta    Tb  U_max  U_min   Kbc
0  1.0   2.0  200.0  0.05  3.0  10.0   -4.0    4.0  10.0
1  2.0   2.0  200.0  0.05  3.0  10.0   -4.0    4.0  10.0
2  3.0   2.0  200.0  0.05  3.0  10.0   -4.0    4.0  10.0
   bus  Type   Tw    T1    T2    T3     T4    Ks  U_smin  U_smax
0  1.0   1.0  2.0  0.25  0.03  0.15  0.015  10.0    -0.1     0.1
1  2.0   1.0  2.0  0.25  0.03  0.15  0.015  10.0    -0.1     0.1
2  3.0   1.0  2.0  0.25  0.03  0.15  0.015  10.0    -0.1     0.1
   bus  Type    K   T1       T2

Exception: 

DP dynamic simulation

In [ ]:
sim_name_dyn_dp = 'DP_only_SYN_case9_dyn'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_dp)

mpc_reader = iaewreader.Reader(mpc_file_path=path_file, mpc_name='case9_static_only_SYN', 
                             mpc_dyn_file_path=path_file, mpc_dyn_name='case9_dyn_static_only_SYN')
mpc_reader.create_dpsim_objects(domain=iaewreader.Domain.DP, frequency=60, with_avr=False, with_tg=False, with_pss=False)
#create a switch
sw = dpsimpy.dp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
switch_closed = 0.2
sw.set_parameters(1e18, switch_closed)
sw.open()
mpc_reader.dpsimpy_comp_dict['Fault'] = [sw]
#Node 11 is the load at 100kV, node 7 ist a node of 400kV grid, between Node 7 and 11 there is a transformer
mpc_reader.dpsimpy_comp_dict['Fault'].append([dpsimpy.dp.SimNode.gnd, mpc_reader.dpsimpy_busses_dict["N11"]])

# create dpsim topology
system_dyn = mpc_reader.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader.system
system_dyn.init_with_powerflow(system_pf)

logger = dpsimpy.Logger(sim_name_dyn_dp)
for node in system_dyn.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)

# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_dyn_dp, dpsimpy.LogLevel.info)
sim.set_system(system_dyn)
sim.set_time_step(1e-3)
sim.set_final_time(10)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.SparseLU)
sim.do_init_from_nodes_and_terminals(True)
sim.do_system_matrix_recomputation(True)
sim.add_logger(logger)

# add event
sw_event_1 = dpsimpy.event.SwitchEvent(0.2, sw, True)
sw_event_2 = dpsimpy.event.SwitchEvent(0.3, sw, False)
sim.add_event(sw_event_1)
sim.add_event(sw_event_2)

sim.run()

EMT dynamic Simulation

In [ ]:
sim_name_dyn_emt = 'EMT_only_SYN_case9_dyn'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_emt)

# read and create dpsim topology
mpc_reader = iaewreader.Reader(mpc_file_path=path_file, mpc_name='case9_static_only_SYN', 
                             mpc_dyn_file_path=path_file, mpc_dyn_name='case9_dyn_static_only_SYN')
mpc_reader.create_dpsim_objects(domain=iaewreader.Domain.EMT, frequency=60, log_level=dpsimpy.LogLevel.info, 
                                 with_avr=False, with_tg=False, with_pss=False)

#Fault at node 11 is the load at 100kV, node 7 ist a node of 400kV grid, between Node 7 and 11 there is a transformer
sw = dpsimpy.emt.ph3.SeriesSwitch('Fault', dpsimpy.LogLevel.info)
switch_closed = 0.2
sw.set_parameters(1e18, switch_closed)
sw.open()
mpc_reader.dpsimpy_comp_dict['Fault'] = [sw]
mpc_reader.dpsimpy_comp_dict['Fault'].append([dpsimpy.emt.SimNode.gnd, mpc_reader.dpsimpy_busses_dict["N11"]])

# create dpsim topology
mpc_reader.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader.system
system_dyn.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

# log results
logger = dpsimpy.Logger(sim_name_dyn_emt)
for node in system_dyn.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
     
# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_dyn_emt, dpsimpy.LogLevel.debug)
sim.set_system(system_dyn)
sim.set_time_step(1e-4)
sim.set_final_time(10)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.SparseLU)
sim.do_init_from_nodes_and_terminals(True)
sim.do_system_matrix_recomputation(True)
sim.add_logger(logger)

# add events
sw_event_1 = dpsimpy.event.SwitchEvent(0.2, sw, True)
sw_event_2 = dpsimpy.event.SwitchEvent(0.3, sw, False)
sim.add_event(sw_event_1)
sim.add_event(sw_event_2)

sim.set_time_step(1e-4)
sim.run()

Read Reslust 

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt

dpsim_result_file = 'logs/' + sim_name_dyn_sp + '/' + sim_name_dyn_sp + '.csv'
ts_dpsim_sp = read_timeseries_csv(dpsim_result_file)
phasors = ts.phasors(ts_dpsim_sp)

dpsim_result_file = 'logs/' + sim_name_dyn_dp + '/' + sim_name_dyn_dp + '.csv'
ts_dpsim_dp = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_dyn_emt + '/' + sim_name_dyn_emt + '.csv'
ts_dpsim_emt = read_timeseries_csv(dpsim_result_file)